<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Общий-вывод" data-toc-modified-id="Общий-вывод-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Общий вывод</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

<b>Описание проекта</b>  

Нам надо защитить данные клиентов страховой компании «Хоть потоп». Требуется разработь такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Надо обосновать корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.  

Наша задача:  
- Загрузить и изучить данные.  
- Ответить на вопрос и обосновать решение:   
  Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии? (Её можно обучить заново.)  
  a. Изменится. Привести примеры матриц.  
  b. Не изменится. Указать, как связаны параметры линейной регрессии в исходной задаче и в преобразованной.  
- Предложить алгоритм преобразования данных для решения задачи. Обосновать, почему качество линейной регрессии не поменяется. 
- Запрограммировать этот алгоритм, применив матричные операции. Проверить, что качество линейной регрессии из sklearn не отличается до и после преобразования. Применить метрику R2.  

Описание данных:   
- Признаки: пол, возраст и зарплата застрахованного, количество членов его семьи.  
- Целевой признак: количество страховых выплат клиенту за последние 5 лет.

## Загрузка данных

In [18]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [19]:
# Читаем данные из csv-файла и сохраним в переменную
data = pd.read_csv('https://code.s3.yandex.net/datasets/insurance.csv')

In [20]:
# Изучим общую информацию о полученном датафрейме
display(data.info(), data.head(), 
        data.describe(include='all').round(2),
        data.corr().style.background_gradient(cmap='PuBuGn'), 
        data.isna().sum(), data.shape)                                    
print('Количество дупликатов:', data.duplicated().sum()) 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


None

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.0,5000.00,5000.00,5000.00,5000.00
mean,0.5,30.95,39916.36,1.19,0.15
std,0.5,8.44,9900.08,1.09,0.46
min,0.0,18.00,5300.00,0.00,0.00
25%,0.0,24.00,33300.00,0.00,0.00
50%,0.0,30.00,40200.00,1.00,0.00
75%,1.0,37.00,46600.00,2.00,0.00
max,1.0,65.00,79000.00,6.00,5.00


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
Пол,1.000000,0.002074,0.014910,-0.008991,0.010140
Возраст,0.002074,1.000000,-0.019093,-0.006692,0.651030
Зарплата,0.014910,-0.019093,1.000000,-0.030296,-0.014963
Члены семьи,-0.008991,-0.006692,-0.030296,1.000000,-0.036290
Страховые выплаты,0.010140,0.651030,-0.014963,-0.036290,1.000000


Пол                  0
Возраст              0
Зарплата             0
Члены семьи          0
Страховые выплаты    0
dtype: int64

(5000, 5)

Количество дупликатов: 153


В таблице 5 столбцов и 5000 строк. Пропусков в данных нет. Все данные выглядят корректно. Нет явно выбивающихся значений. Имеется высокая зависимость признаков 'Страховые выплаты' и 'Возраст'. Таблица содержит 153 явных дупликата. Т.к. нет уникальных идентификаторов клиентов, мы можем предположить, что данные могут совпадать полностью у разных клиентов. Оставим эти данные без изменений. 

## Умножение матриц

Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии? (Её можно обучить заново.)

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** Не изменится

**Обоснование:**   

Запишем новое представление матрицы признаков с учетом умнажения на обратную матрицу:  

$$ B = XP $$  

Тогда формулы c учетом новых введений будут иметь следующий вид:  

Предсказания:  $$ A = BW $$  

Формула обучения:  $$ W = (B^TB)^{-1}B^Ty $$, где W - вектор весов  

Подставим значение $ B $ в формулу обучения и преобразуем ее:  


$$ W = ((XP)^T(XP))^{-1}(XP)^T y $$  

$$ W = (P^T X^T X P)^{-1} P^T X^T y $$   

$$ W = (P^T(X^TX)P)^{-1}P^T X^T y $$   

$$ W = ((X^TX)P)^{-1} P^{T^{-1}} P^T X^T y $$ 


По свойству обратной матрицы - при умножение матрицы на обратную ей получается единичная матрица:  

$$ (P^T)^{-1}  P^T = E $$  

Вектор весов линейной регрессии имеет следующий вид:    

$$ W = ((X^TX)P)^{-1} E X^T y $$ 


Так как при умножение любой матрицы на единичную матрицу получается таже самая матрица, получаем:  

$$ W = P^{-1}(X^T X)^{-1}  X^T y $$    

Полученную выше формулу можно представить в виде: $$ W=P^{-1}w $$, так как по условию: $$ w = (X^T X)^{-1} X^T y $$  

Подставим полученное значение вектора весов линейной регрессии в формулу предсказаний:  

$$ A = B (P^{-1}w) $$  

$$ A = XP (P^{-1}w) $$  

По свойству обратной матрицы и еденичной матрицы сократим $ P $  и $ P^{-1} $ :  

$$ A = X w $$



Таким образом, мы получили аналогичную формулу предсказаний, что и была в начале, значит умножение признаков на обратимую матрицу не влияет на предсказания модели.

## Алгоритм преобразования

**Алгоритм**

В качестве алгоритма защиты данных клиентов страховой компании будем использовать умножение матрицы признаков $ X $ на случайную обратимую матрицу $ P $ размером 4х4. Создадим модель линейной регрессии для исходных признаков и посчитаем метрику R2. Далее умножим исходные признаки на обратимую матрицу и опять посчитаем метрику R2. Умножение признаков на обратимую матрицу не приводит к изменению предсказаний, поэтому результаты R2 должны оказаться равны.

**Обоснование**

Необходимо создать обратимую матрицу размером 4х4. При этом важно не создать случайную необратимую матрицу, т.к. это нарушит наш алгоритм. 

In [21]:
# Создадим рандомную матрицу 4х4
rng = np.random.default_rng(seed=12345)
P = rng.random((4, 4))
P

array([[0.22733602, 0.31675834, 0.79736546, 0.67625467],
       [0.39110955, 0.33281393, 0.59830875, 0.18673419],
       [0.67275604, 0.94180287, 0.24824571, 0.94888115],
       [0.66723745, 0.09589794, 0.44183967, 0.88647992]])

In [22]:
# Проверим матрицу на обратимость
np.linalg.inv(P)

array([[-1.97240014,  1.76004024, -0.08309671,  1.22285233],
       [ 0.14111106,  0.32873452,  1.02824721, -1.27752175],
       [ 0.8908452 ,  0.90302415, -0.59501472, -0.23290483],
       [ 1.02530945, -1.81039816,  0.24787878,  0.46192295]])

Сообщения об ошибке нет, значит матрица обратима. 

In [23]:
# Разобьем данные data на признаки и целевой признак, сохраним в отдельных переменных
features = data.drop('Страховые выплаты', axis=1)
target = data['Страховые выплаты']

In [24]:
# Выведем на экран размеры этих выборок
print(features.shape)
print(target.shape)

(5000, 4)
(5000,)


Применим формулы: 
$$ a = Xw $$
$$ w = (X^T X)^{-1} X^T y $$

In [25]:
# Рассчитаем результаты для исходных признаков
w = np.linalg.inv(features.T.dot(features)).dot(features.T).dot(target)
a = features @ w

In [26]:
# Умножим исходные признаки на нашу случайную матрицу
features_random = features @ P
# Выведем на экран размер выборки
print(features_random.shape)

(5000, 4)


In [27]:
# Рассчитаем результаты для признаков, умноженных на обратимую матрицу
w2 = np.linalg.inv(features_random.T.dot(features_random)).dot(features_random.T).dot(target)
a2 = features_random @ w2

In [28]:
# Определим разницу между полученными результатами
print(round((a - a2).sum(),2))

0.0


Выбранный нами алгоритм защиты данных клиентов не создает разницы в предсказаниях при использовании исходных признаков и при использовании преобразованных признаков. Качество модели линейной регрессии не поменяется.  

## Проверка алгоритма

In [29]:
# Обучим модель линейной регрессии для исходных признаков
model1 = LinearRegression()
model1.fit(features, target)
predictions1 = model1.predict(features)
# Вычислим результат метрики для исходных признаков
R2 = r2_score(target, predictions1)
print(f'Метрика R2 при использовании исходных признаков - {round((R2),3)}')

Метрика R2 при использовании исходных признаков - 0.425


In [30]:
# Обучим модель линейной регрессии для признаков, умноженных на обратимую матрицу
model2 = LinearRegression()
model2.fit(features_random, target)
predictions2 = model2.predict(features_random)
# Вычислим результат метрики для признаков, умноженных на обратимую матрицу
R2_random = r2_score(target, predictions2)
print(f'Метрика R2 при использовании признаков, умноженых на обратимую матрицу - {round((R2_random),3)}')

Метрика R2 при использовании признаков, умноженых на обратимую матрицу - 0.425


Метрики R2 обеих моделей одиннаковые, следовательно умножение исходных данных на случайную обратимую матрицу может быть применено в качестве алгоритма защиты данные клиентов страховой компании.

##  Общий вывод

Было проведено исследование, чтобы защитить данные клиентов страховой компании «Хоть потоп». Требовалось разработь такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию.  
В ходе работы были изучены предоставленные данные. Таблица состоит из 5 столбцов и 5000 строк. Пропусков в данных нет. Все данные выглядят корректно. Нет явно выбивающихся значений. Имеется высокая зависимость признаков 'Страховые выплаты' и 'Возраст'. Таблица содержит 153 явных дупликата. Т.к. нет уникальных идентификаторов клиентов, мы предположили, что данные могут совпадать полностью у разных клиентов. Оставили эти данные без изменений.  
Было обосновано, что при умножении признаков на обратимую матрицу, качество модели линейной регрессии не меняется.  
В качестве алгоритма защиты данных клиентов страховой компании использовали умножение матрицы признаков 𝑋 на случайную обратимую матрицу 𝑃 размером 4х4. Создали модель линейной регрессии для исходных признаков и посчитали метрику R2. Далее умножили исходные признаки на обратимую матрицу, провели машинное обучение с помощью алгоритма линейной регрессии и опять посчитали метрику R2. Метрики R2 обеих моделей одиннаковые, следовательно умножение исходных данных на случайную обратимую матрицу может быть применено в качестве алгоритма защиты данные клиентов страховой компании.  
